Packages

In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt # didn't use, can be deleted
import tensorflow as tf
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [2]:
import nltk
#seperating words
nltk.download('punkt_tab')
#removing insignificant words
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\danni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Data Cleaning

In [3]:
#load data from csv
emotions_data = pd.read_csv("emotions.csv", encoding = "UTF-8")
emotions_data.head(5)

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [4]:
#shape of emotion data
emotions_data.shape

(416809, 2)

In [5]:
#check for missing values in dataset
emotions_data.isnull().sum()

text     0
label    0
dtype: int64

In [6]:
#check distribution of 'label' column
emotions_data['label'].value_counts()

label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64

In [7]:
#downsample
min_count = emotions_data['label'].value_counts().min()
selected_cols = ['label'] + [col for col in emotions_data.columns if col != 'label']
emotions_data = emotions_data.groupby('label')[selected_cols].apply(lambda x: x.sample(min_count, random_state=18)).reset_index(drop=True)
emotions_data['label'].value_counts()

label
0    14972
1    14972
2    14972
3    14972
4    14972
5    14972
Name: count, dtype: int64

In [8]:
emotions_data.head(10)

,label,text
0,0,i always feel like abby is being punished for ...
1,0,i used to feel devastated when someone critici...
2,0,i feel like i ve lost someone utterly irreplac...
3,0,i go and feel morose about this
4,0,i had been feeling out my more submissive side...
5,0,i lacked just reinforced my feelings of guilt ...
6,0,i was for the first time since i came over her...
7,0,i had so many peo ple to be grate ful for so w...
8,0,i love the sunset because after a tiring dayti...
9,0,im ashamed to admit it but i do have that comp...


Word2Vec Word Embedding

In [9]:
#loads pretrained model for vectorizing sentences, Word2Vec
path = "GoogleNews-vectors-negative300.bin.gz"
model = KeyedVectors.load_word2vec_format(path, binary=True)
vocab_size = len(model.index_to_key)

In [10]:
meaningless_bank = [ # stop word bank
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", 
    "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
    "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", 
    "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", 
    "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", 
    "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", 
    "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", 
    "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", 
    "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", 
    "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", 
    "don", "should", "now", "uh", "um", "well", "like", "okay", "yeah", "oh", "ah", "er", "hm", "hmm"
]

In [11]:
#method for mean pooling
tokenizer = nltk.RegexpTokenizer(r'\w+')
def embedSentence_meanPooling(row):
    #uses nltk to seperate sentence into words omitting punctuation
    tokens = tokenizer.tokenize(row.lower())
    #averages all token vectors
    # vector = np.array([0]*300)
    result = [np.array([0]*300) for _ in range(100)]
    count = 0
    for i in tokens:
        if i in meaningless_bank: # skip the words that meaningless, avoid bias
            continue
        if i in model: #only adds tokens present in the model
            result[tokens.index(i)] = model.get_vector(i)
            # vector = vector + model.get_vector(i)
            # count += 1
    # vector = vector/max(1, count)
    # return vector
    return result

In [12]:
#embeds all x from word to values
# emotions_data["text"] = emotions_data["text"].apply(embedSentence_meanPooling)
emotions_data_3D = emotions_data["text"].apply(embedSentence_meanPooling)

In [13]:
#set input and output
# X = emotions_data["text"].values

X = emotions_data_3D
y = emotions_data["label"].values

In [15]:
#compress X into batch dimension vector to feed into tensorflow
# X = np.stack(X).astype(np.float32)
X = [np.stack(i).astype(np.float32) for i in X]
# X.shape

In [16]:
#split x, y into training
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=0) # we need talk about this part
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=0)

In [30]:
len(X_train[0])

100

In [31]:
# covn_base = tf.keras.applications.Xception(
#     include_top=False,
#     input_shape=(100,300,3),
#     pooling='avg',
#     classifier_activation='softmax'
# )

# covn_base.trainable = False

In [32]:
# covn_base.summary()

In [36]:
#builds tensorflow model
tf_model = tf.keras.Sequential([
                                # covn_base,
                                # tf.keras.layers.Input(shape=(100,300)),
                                tf.keras.layers.Dense(256, activation='relu'),
                                # tf.keras.layers.Dropout(0.2), # let each layer randomly forget some data, prevent overfitting
                                # tf.keras.layers.BatchNormalization(), #  normalizes the activations (outputs) of the previous layer, make the training more stable
                                tf.keras.layers.Dense(6, activation='softmax')
                                ])

In [34]:
tf_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                loss='sparse_categorical_crossentropy',
                metrics=["accuracy"])

In [35]:
tf_model.evaluate(X_train, y_train)
# tf_model.evaluate(X, y) # using the original data is fine

ValueError: Data cardinality is ambiguous:
  x sizes: 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 1

In [28]:
tf_model.fit(X_train, y_train, batch_size=24, epochs = 10, validation_data=(X_valid, y_valid))
# tf_model.fit(X, y, batch_size=32, epochs = 30, validation_data=(X_valid, y_valid)) # we need talk about this part

ValueError: Data cardinality is ambiguous:
  x sizes: 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 1

In [ ]:
#test on one sample sentence: sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5)
sample_sentence = np.stack(embedSentence_meanPooling("god i would happy dying right now")).astype(np.float32)
sample_sentence = np.expand_dims(sample_sentence, axis=0)   
predicted = np.argmax(tf_model.predict(sample_sentence), axis=1)
predicted